# LLMs and LlamaIndex ◦ April 4 2024 ◦ Vector Institute Prompt Engineering Lab

In [ ]:
##################################################################
# Venue: Vector Institute Prompt Engineering Lab
# Talk: Building RAG Systems with LlamaIndex
# Speaker: Andrei Fajardo
##################################################################

![Title Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/title.excalidraw.svg)

![Title Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/framework.excalidraw.svg)

#### Notebook Setup & Dependency Installation

In [ ]:
%pip install llama-index-vector-stores-qdrant -q

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
!mkdir data
!wget "https://vectorinstitute.ai/wp-content/uploads/2024/02/Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf" -O "./data/Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf"

mkdir: data: File exists
--2024-04-04 11:34:26--  https://vectorinstitute.ai/wp-content/uploads/2024/02/Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf
Resolving vectorinstitute.ai (vectorinstitute.ai)... 162.159.134.42
Connecting to vectorinstitute.ai (vectorinstitute.ai)|162.159.134.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5305903 (5.1M) [application/pdf]
Saving to: ‘./data/Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf’

./data/Vector-Annua 100%[===================>]   5.06M  14.2MB/s    in 0.4s    

2024-04-04 11:34:27 (14.2 MB/s) - ‘./data/Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf’ saved [5305903/5305903]



## Motivation

![Motivation Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/motivation.excalidraw.svg)


In [ ]:
# query an LLM and ask it about Mistplay
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4-turbo-preview")

response = llm.complete("What is Vector Institute all about?")

In [ ]:
print(response)

The Vector Institute is a research institution in Toronto, Canada, dedicated to the field of artificial intelligence (AI), with a particular focus on machine learning and deep learning. It was established in March 2017 as part of Canada's strategy to lead in the AI domain, leveraging the country's significant contributions to the development of AI technologies.

The institute's mission revolves around several key objectives:

1. **Research Excellence**: Vector aims to attract, retain, and develop top talent in the AI field. It fosters an environment where leading researchers and industry experts can collaborate on cutting-edge projects, pushing the boundaries of what AI can achieve.

2. **Industry Collaboration**: The institute works closely with companies across various sectors, helping them to adopt AI technologies to drive innovation and competitiveness. This includes providing guidance on AI strategy, development, and deployment.

3. **Talent Development**: Recognizing the critical

In [ ]:
response = llm.complete(
    "According to Vector Institute's Annual Report 2022-2023, "
    "how many AI jobs were created in Ontario?"
)

In [ ]:
print(response)

As of my last update in April 2023, I don't have access to the specific details of the Vector Institute's Annual Report for 2022-2023. Therefore, I cannot provide the exact number of AI jobs created in Ontario as reported in that document. For the most accurate and up-to-date information, I recommend checking the Vector Institute's official website or contacting them directly. They may have published the report or related summaries that include the data you're interested in.


## Basic RAG in 3 Steps

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/subheading.excalidraw.svg)


1. Build external knowledge (i.e., uploading updated data sources)
2. Retrieve
3. Augment and Generate

## 1. Build External Knowledge

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/step1.excalidraw.svg)

In [ ]:
"""Load the data.

With llama-index, before any transformations are applied,
data is loaded in the `Document` abstraction, which is
a container that holds the text of the document.
"""

from llama_index.core import SimpleDirectoryReader

loader = SimpleDirectoryReader(input_dir="./data")
documents = loader.load_data()

In [ ]:
# if you want to see what the text looks like
documents[1].text

'Vector Institute   |   Annual Report 2022-2023      2\nTABLE OF CONTENTS\nMESSAGE FROM THE BOARD CHAIR  3 \nMESSAGE FROM THE PRESIDENT AND CEO 4\nVECTOR’S VISION AND MISSION 5\nONTARIO’S VIBRANT AI ECOSYSTEM 6\nINDUSTRY INNOVATION   7\nRESEARCH AND EDUCATION  1 4TALENT AND WORKFORCE DEVELOPMENT 24\nHEALTH   3 1\nAI ENGINEERING   36\nTHOUGHT LEADERSHIP  39\nFINANCIALS 42\n'

In [ ]:
"""Chunk, Encode, and Store into a Vector Store.

To streamline the process, we can make use of the IngestionPipeline
class that will apply your specified transformations to the
Document's.
"""

from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

client = qdrant_client.QdrantClient(location=":memory:")
vector_store = QdrantVectorStore(client=client, collection_name="test_store")

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store,
)
_nodes = pipeline.run(documents=documents, num_workers=4)

In [ ]:
# if you want to see the nodes
# len(_nodes)
_nodes[1].text

'Vector Institute   |   Annual Report 2022-2023      2\nTABLE OF CONTENTS\nMESSAGE FROM THE BOARD CHAIR  3 \nMESSAGE FROM THE PRESIDENT AND CEO 4\nVECTOR’S VISION AND MISSION 5\nONTARIO’S VIBRANT AI ECOSYSTEM 6\nINDUSTRY INNOVATION   7\nRESEARCH AND EDUCATION  1 4TALENT AND WORKFORCE DEVELOPMENT 24\nHEALTH   3 1\nAI ENGINEERING   36\nTHOUGHT LEADERSHIP  39\nFINANCIALS 42'

In [ ]:
"""Create a llama-index... wait for it... Index.

After uploading your encoded documents into your vector
store of choice, you can connect to it with a VectorStoreIndex
which then gives you access to all of the llama-index functionality.
"""

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

## 2. Retrieve Against A Query

![Step2 Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/step2.excalidraw.svg)

In [ ]:
"""Retrieve relevant documents against a query.

With our Index ready, we can now query it to
retrieve the most relevant document chunks.
"""

retriever = index.as_retriever(similarity_top_k=2)
retrieved_nodes = retriever.retrieve(
    "According to Vector Institute's Annual Report 2022-2023, "
    "how many AI jobs were created in Ontario?"
)

In [ ]:
# to view the retrieved nodes
retrieved_nodes

[NodeWithScore(node=TextNode(id_='af5b8613-9990-492f-8a0c-94ced4be3c61', embedding=None, metadata={'page_label': '6', 'file_name': 'Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf', 'file_path': '/Users/nerdai/talks/2024/vector-pe-lab/data/Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf', 'file_type': 'application/pdf', 'file_size': 5305903, 'creation_date': '2024-04-04', 'last_modified_date': '2024-02-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='47692fff-7f25-4472-846f-c3f8d30a16e5', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '6', 'file_name': 'Vector-Annual-Report-2022-23_accessible_rev0224-1.pdf', 'file_path': '/Users/nerdai/talks/2024/vector-pe-lab/data/Vector-Ann

## 3. Generate Final Response

![Step3 Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/step3.excalidraw.svg)

In [ ]:
"""Context-Augemented Generation.

With our Index ready, we can create a QueryEngine
that handles the retrieval and context augmentation
in order to get the final response.
"""

query_engine = index.as_query_engine()

In [ ]:
# to inspect the default prompt being used
print(
    query_engine.get_prompts()[
        "response_synthesizer:text_qa_template"
    ].default_template.template
)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


In [ ]:
response = query_engine.query(
    "According to Vector Institute's Annual Report 2022-2023, "
    "how many AI jobs were created in Ontario?"
)
print(response)

20,634 AI jobs were created in Ontario according to Vector Institute's Annual Report 2022-2023.


## More Queries

### Comparisons

In [ ]:
query = (
    "According to Vector Institute's 2022-2023 annual report, "
    "how many new AI companies were established in Ontario?"
)

response = query_engine.query(query)
print(response)

27


In [ ]:
query = (
    "According to Vector Institute's 2022-2023 annual report, "
    "what was the dollar value for Unrestricted net assets in "
    "the years 2022 & 2023?"
)

response = query_engine.query(query)
print(response)

In the years 2022 and 2023, the dollar value for Unrestricted net assets was $51,630,471 and $52,937,983, respectively.


In [ ]:
query = (
    "According to Vector Institute's 2022-2023 annual report, "
    "what companies were platinum sponsors?"
)

response = query_engine.query(query)
print(response)

According to Vector Institute's 2022-2023 annual report, the platinum sponsors were BMO Financial Group, Google, Loblaw Companies Ltd., NVIDIA, RBC, Scotiabank, Shopify Inc., and TD Bank Group.


## In Summary

- LLMs as powerful as they are, don't perform too well with knowledge-intensive tasks (domain specific, updated data, long-tail)
- Context augmentation has been shown (in a few studies) to outperform LLMs without augmentation
- In this notebook, we showed one such example that follows that pattern.

## Storing Results In Structured Objects

![DataExtractions](https://media.licdn.com/dms/image/D4E22AQGwPmZ5RRhbyA/feedshare-shrink_1280/0/1711823067172?e=1715212800&v=beta&t=fJtksPZ3Fm-BOrKRCwa6BrYyuxlNFDuop3ZSopMl53M)

In [ ]:
import json
from llama_index.core.bridge.pydantic import BaseModel, Field
from typing import List

from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.llms.openai import OpenAI

### Sponsors

In [ ]:
class VectorSponsors(BaseModel):
    """Data model for Vector Institute sponsors 2022-2023."""

    platinum: str = Field(description="Platinum sponsors")
    gold: str = Field(description="Gold sponsors")
    silver: str = Field(description="Silver sponsors")
    bronze: List[str] = Field(description="Bronze sponsors")

In [ ]:
prompt_template_str = """\
Here is the 2022-2023 Annual Report for Vector Institute:
{document_text}
Provide the names sponsor companies according to their tiers.
"""

program = OpenAIPydanticProgram.from_defaults(
    output_cls=VectorSponsors,
    prompt_template_str=prompt_template_str,
    llm=OpenAI("gpt-4-turbo-preview"),
    verbose=True,
)

In [ ]:
report_as_text = ""
for d in documents:
    report_as_text += d.text

In [ ]:
sponsors = program(document_text=report_as_text)

Function call: VectorSponsors with args: {"platinum":"BMO Financial Group, Google, Loblaw Companies Ltd., NVIDIA, RBC, Scotiabank, Shopify Inc., TD Bank Group, Thomson Reuters","gold":"Accenture, Air Canada, Bell Canada, Boehringer Ingelheim (Canada) Ltd., Canadian Tire Corporation, Ltd., CIBC, CN, Deloitte Canada, EY Canada, Georgian, KPMG Canada, KT Corporation, Magna International, OMERS, PwC Canada, Roche Canada, Sun Life Financial, TELUS, Thales Canada","silver":"EllisDon Corporation, Linamar Corporation","bronze":["Ada","ALS","GoldSpot Discoveries Ltd.","AltaML","Avidbots","BenchSci","Blue J","Canvass Analytics Inc.","Clearpath","Cohere","Cyclica","Darwin AI","Deep Genomics","FreshBooks","Integrate.ai","Layer 6","League","MindBridge Analytics Inc.","Private AI","Riskfuel","Shakudo","Signal 1","Stradigi AI","Surgical Safety Technologies","TealBook","Troj.AI","Wysdom AI"]}


In [ ]:
print(json.dumps(sponsors.dict(), indent=4))

{
    "platinum": "BMO Financial Group, Google, Loblaw Companies Ltd., NVIDIA, RBC, Scotiabank, Shopify Inc., TD Bank Group, Thomson Reuters",
    "gold": "Accenture, Air Canada, Bell Canada, Boehringer Ingelheim (Canada) Ltd., Canadian Tire Corporation, Ltd., CIBC, CN, Deloitte Canada, EY Canada, Georgian, KPMG Canada, KT Corporation, Magna International, OMERS, PwC Canada, Roche Canada, Sun Life Financial, TELUS, Thales Canada",
    "silver": "EllisDon Corporation, Linamar Corporation",
    "bronze": [
        "Ada",
        "ALS",
        "GoldSpot Discoveries Ltd.",
        "AltaML",
        "Avidbots",
        "BenchSci",
        "Blue J",
        "Canvass Analytics Inc.",
        "Clearpath",
        "Cohere",
        "Cyclica",
        "Darwin AI",
        "Deep Genomics",
        "FreshBooks",
        "Integrate.ai",
        "Layer 6",
        "League",
        "MindBridge Analytics Inc.",
        "Private AI",
        "Riskfuel",
        "Shakudo",
        "Signal 1",
        

### Useful links

[website](https://www.llamaindex.ai/) ◦ [llamahub](https://llamahub.ai) ◦ [llamaparse](https://github.com/run-llama/llama_parse) ◦ [github](https://github.com/run-llama/llama_index) ◦ [medium](https://medium.com/@llama_index) ◦ [rag-bootcamp-poster](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/final_poster.excalidraw.svg)